In [1]:
import requests
import pandas as pd
from multiprocessing import Pool
from progressbar import ProgressBar
from unidecode import unidecode

In [2]:
call = 'http://www.federalregister.gov/api/v1/articles.json?fields%5B%5D=raw_text_url&fields%5B%5D=title&fields%5B%5D=toc_subject&fields%5B%5D=topics&per_page=1000&order=oldest&conditions%5Bpublication_date%5D%5Bgte%5D=2013-01-01&conditions%5Bagencies%5D%5B%5D=commerce-department&conditions%5Bagencies%5D%5B%5D=state-department&conditions%5Bagencies%5D%5B%5D=treasury-department'
r = requests.get(call)

In [3]:
# bug where api only returns last 10,000 documents in spite of there being 10,877 within the search range
# fork api and correct

print r.json()['description']
print r.json()['count']
df = pd.DataFrame(r.json()['results'])

Documents published on or after 01/01/2013 and from Commerce Department, State Department, or Treasury Department
10877


In [4]:
while 'next_page_url' in r.json():
    next_page = r.json()['next_page_url']
    r = requests.get(next_page)
    print r
    df = pd.concat([df, pd.DataFrame(r.json()['results'])], ignore_index=True)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [5]:
def grab_text(url):
    r = requests.get(url)
    r.encoding = 'utf-8'
    text = r.text
    return text

In [7]:
df.head()

,raw_text_url,title,toc_subject,topics
0,https://www.federalregister.gov/articles/text/...,Culturally Significant Objects Imported for Ex...,Culturally Significant Objects Imported for Ex...,[]
1,https://www.federalregister.gov/articles/text/...,Proposed Collection: Comment Request,None,[]
2,https://www.federalregister.gov/articles/text/...,National Medal of Technology and Innovation Ca...,Calls for Nominations:,[]
3,https://www.federalregister.gov/articles/text/...,Additional Designations of Individuals Pursuan...,Blocking or Unblocking of Persons and Property:,[]
4,https://www.federalregister.gov/articles/text/...,Fisheries of the Northeastern United States; S...,Fisheries of the Northeastern United States:,[]


In [8]:
df.shape

(10000, 4)

In [9]:
pool = Pool()
results = pool.map(grab_text, df.raw_text_url.tolist())
pool.close()

In [10]:
df['raw_text'] = results

In [11]:
df.to_pickle('docs_df')

In [6]:
r = requests.get('http://www.federalregister.gov/api/v1/agencies')

In [15]:
agencies = pd.DataFrame(r.json())[['name', 'short_name', 'description']]

In [17]:
agencies.to_pickle('agencies_df')